In [ ]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
# from mlfs import config
# settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/fredrikstrom/Documents/KTH_Dokument/Scalable ML/labs/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [98]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [ ]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
locations_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
locations = json.loads(locations_str) # Change 

country=locations[0]['country']
city=locations[0]['city']

today = datetime.date.today()

locations_str

2025-11-18 11:19:58,483 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-18 11:19:58,487 INFO: Initializing external client
2025-11-18 11:19:58,488 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 11:20:00,023 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286301


'[{"station_id": 6037, "country": "germany", "city": "munich", "street": "Munich", "aqicn_url": "https://api.waqi.info/feed/@6037", "latitude": 48.1351253, "longitude": 11.5819806}, {"station_id": 6041, "country": "germany", "city": "munich", "street": "Johanneskirchen, M\\u00fcnchen, Germany", "aqicn_url": "https://api.waqi.info/feed/@6041", "latitude": 48.169444, "longitude": 11.65}, {"station_id": 6042, "country": "germany", "city": "munich", "street": "Landshuter Allee, M\\u00fcnchen, Germany", "aqicn_url": "https://api.waqi.info/feed/@6042", "latitude": 48.1552945, "longitude": 11.5367712}, {"station_id": 6043, "country": "germany", "city": "munich", "street": "Lothstra\\u00dfe, M\\u00fcnchen, Germany", "aqicn_url": "https://api.waqi.info/feed/@6043", "latitude": 48.1542256, "longitude": 11.5537444}, {"station_id": 6044, "country": "germany", "city": "munich", "street": "Stachus, M\\u00fcnchen, Germany", "aqicn_url": "https://api.waqi.info/feed/@6044", "latitude": 48.1390293, "lon

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [ ]:
#Retrieve feature groups
# air_quality_fg = fs.get_feature_group(
#     name='air_quality',
#     version=3,
# ) #--- C-level grade ---

aq_fg_munich = fs.get_feature_group(
    name='air_quality_munich',
    version=1,
) # --- A-level feature group with multiple sensors ---

weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


## Grade C task 

In [94]:
#aq_lag_features = air_quality_fg.read() #--- C-level grade ---
aq_lag_features = aq_fg_munich.read() # --- A-level feature group with multiple sensors ---
aq_lag_features = aq_lag_features.sort_values(["station_id", "date"])

# Last 3 rows for *each* sensor
last_3 = aq_lag_features.groupby("station_id").tail(3)
last_3

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.05s) 


,date,pm25,station_id,street,country,city,url,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d
2659,2025-11-15 00:00:00+00:00,69.0,6037,Munich,germany,munich,https://aqicn.org/station/Munich,62.0,50.0,50.0
1100,2025-11-16 00:00:00+00:00,47.0,6037,Munich,germany,munich,https://aqicn.org/station/Munich,69.0,62.0,50.0
349,2025-11-17 00:00:00+00:00,34.0,6037,Munich,germany,munich,https://aqicn.org/station/Munich,47.0,69.0,62.0
2624,2025-11-15 00:00:00+00:00,61.0,6041,"Johanneskirchen, München, Germany",germany,munich,"https://aqicn.org/station/Johanneskirchen, Mün...",43.0,34.0,42.0
2536,2025-11-16 00:00:00+00:00,38.0,6041,"Johanneskirchen, München, Germany",germany,munich,"https://aqicn.org/station/Johanneskirchen, Mün...",61.0,43.0,34.0
1543,2025-11-17 00:00:00+00:00,33.0,6041,"Johanneskirchen, München, Germany",germany,munich,"https://aqicn.org/station/Johanneskirchen, Mün...",38.0,61.0,43.0
4103,2025-11-15 00:00:00+00:00,55.0,6042,"Landshuter Allee, München, Germany",germany,munich,"https://aqicn.org/station/Landshuter Allee, Mü...",58.0,47.0,48.0
4288,2025-11-16 00:00:00+00:00,41.0,6042,"Landshuter Allee, München, Germany",germany,munich,"https://aqicn.org/station/Landshuter Allee, Mü...",55.0,58.0,47.0
3602,2025-11-17 00:00:00+00:00,41.0,6042,"Landshuter Allee, München, Germany",germany,munich,"https://aqicn.org/station/Landshuter Allee, Mü...",41.0,55.0,58.0
1687,2025-11-15 00:00:00+00:00,55.0,6043,"Lothstraße, München, Germany",germany,munich,"https://aqicn.org/station/Lothstraße, München,...",42.0,39.0,40.0


In [101]:
dfs = []

for loc in locations:
    aqicn_url = loc["aqicn_url"]
    street    = loc["street"]
    station_id = loc["station_id"]
    latitude  = loc["latitude"]
    longitude = loc["longitude"]

    try:
        df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
    except hopsworks.RestAPIError:
        print(f"Failed for station_id={station_id}, street={street}")
        continue

    df["station_id"] = station_id
    df["latitude"]   = latitude
    df["longitude"]  = longitude
    dfs.append(df)

aq_today_df = pd.concat(dfs, ignore_index=True)
aq_today_df

,pm25,country,city,street,date,url,station_id,latitude,longitude
0,NaN,germany,munich,Munich,2025-11-18,https://api.waqi.info/feed/@6037,6037,48.135125,11.581981
1,17.0,germany,munich,"Johanneskirchen, München, Germany",2025-11-18,https://api.waqi.info/feed/@6041,6041,48.169444,11.650000
2,34.0,germany,munich,"Landshuter Allee, München, Germany",2025-11-18,https://api.waqi.info/feed/@6042,6042,48.155294,11.536771
3,25.0,germany,munich,"Lothstraße, München, Germany",2025-11-18,https://api.waqi.info/feed/@6043,6043,48.154226,11.553744
4,21.0,germany,munich,"Stachus, München, Germany",2025-11-18,https://api.waqi.info/feed/@6044,6044,48.139029,11.565558


In [102]:
# Ensure both are timezone-aware UTC
last_3["date"] = pd.to_datetime(last_3["date"], utc=True)
aq_today_df["date"] = pd.to_datetime(aq_today_df["date"], utc=True)

combined_aq_df = pd.concat([last_3, aq_today_df], ignore_index=True)
combined_aq_df = combined_aq_df.sort_values(["station_id", "date"])

combined_aq_df = combined_aq_df.drop(
    columns=[c for c in combined_aq_df.columns if c.startswith("pm25_lag_")],
    errors="ignore",
)

combined_aq_df["pm25_lag_1d"] = (
    combined_aq_df.groupby("station_id")["pm25"].shift(1).astype("float64")
)
combined_aq_df["pm25_lag_2d"] = (
    combined_aq_df.groupby("station_id")["pm25"].shift(2).astype("float64")
)
combined_aq_df["pm25_lag_3d"] = (
    combined_aq_df.groupby("station_id")["pm25"].shift(3).astype("float64")
)

aq_today_with_lags = combined_aq_df.groupby("station_id").tail(1).copy()
aq_today_with_lags

,date,pm25,station_id,street,country,city,url,latitude,longitude,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d
15,2025-11-18 00:00:00+00:00,NaN,6037,Munich,germany,munich,https://api.waqi.info/feed/@6037,48.135125,11.581981,34.0,47.0,69.0
16,2025-11-18 00:00:00+00:00,17.0,6041,"Johanneskirchen, München, Germany",germany,munich,https://api.waqi.info/feed/@6041,48.169444,11.650000,33.0,38.0,61.0
17,2025-11-18 00:00:00+00:00,34.0,6042,"Landshuter Allee, München, Germany",germany,munich,https://api.waqi.info/feed/@6042,48.155294,11.536771,41.0,41.0,55.0
18,2025-11-18 00:00:00+00:00,25.0,6043,"Lothstraße, München, Germany",germany,munich,https://api.waqi.info/feed/@6043,48.154226,11.553744,23.0,36.0,55.0
19,2025-11-18 00:00:00+00:00,21.0,6044,"Stachus, München, Germany",germany,munich,https://api.waqi.info/feed/@6044,48.139029,11.565558,33.0,45.0,66.0


In [107]:
combined_aq_df.drop(columns=['latitude', 'longitude'], inplace=True)
combined_aq_df.head(10)

,date,pm25,station_id,street,country,city,url,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d
0,2025-11-15 00:00:00+00:00,69.0,6037,Munich,germany,munich,https://aqicn.org/station/Munich,NaN,NaN,NaN
1,2025-11-16 00:00:00+00:00,47.0,6037,Munich,germany,munich,https://aqicn.org/station/Munich,69.0,NaN,NaN
2,2025-11-17 00:00:00+00:00,34.0,6037,Munich,germany,munich,https://aqicn.org/station/Munich,47.0,69.0,NaN
15,2025-11-18 00:00:00+00:00,NaN,6037,Munich,germany,munich,https://api.waqi.info/feed/@6037,34.0,47.0,69.0
3,2025-11-15 00:00:00+00:00,61.0,6041,"Johanneskirchen, München, Germany",germany,munich,"https://aqicn.org/station/Johanneskirchen, Mün...",NaN,NaN,NaN
4,2025-11-16 00:00:00+00:00,38.0,6041,"Johanneskirchen, München, Germany",germany,munich,"https://aqicn.org/station/Johanneskirchen, Mün...",61.0,NaN,NaN
5,2025-11-17 00:00:00+00:00,33.0,6041,"Johanneskirchen, München, Germany",germany,munich,"https://aqicn.org/station/Johanneskirchen, Mün...",38.0,61.0,NaN
16,2025-11-18 00:00:00+00:00,17.0,6041,"Johanneskirchen, München, Germany",germany,munich,https://api.waqi.info/feed/@6041,33.0,38.0,61.0
6,2025-11-15 00:00:00+00:00,55.0,6042,"Landshuter Allee, München, Germany",germany,munich,"https://aqicn.org/station/Landshuter Allee, Mü...",NaN,NaN,NaN
7,2025-11-16 00:00:00+00:00,41.0,6042,"Landshuter Allee, München, Germany",germany,munich,"https://aqicn.org/station/Landshuter Allee, Mü...",55.0,NaN,NaN


In [108]:
aq_fg_munich.insert(combined_aq_df)

2025-11-18 11:29:35,325 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286301/fs/1273931/fg/1718942


Uploading Dataframe: 100.00% |██████████| Rows 20/20 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_munich_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286301/jobs/named/air_quality_munich_1_offline_fg_materialization/executions


(Job('air_quality_munich_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 759039
         }
       },
       "result": {
         "observed_value": 17.0,
         "element_count": 20,
         "missing_count": 1,
         "missing_percent": 5.0
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T10:29:35.000325Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expe

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [110]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 48.25°N 11.5°E
Elevation 523.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,2.65,0.0,1.080000,90.000000,munich
1,2025-11-19,4.75,0.0,4.896529,162.897186,munich
2,2025-11-20,1.60,0.3,10.144082,297.474335,munich
3,2025-11-21,-0.90,0.4,11.792404,31.263678,munich
4,2025-11-22,-2.10,0.0,1.609969,333.435028,munich
5,2025-11-23,-3.30,0.0,11.165805,88.152428,munich
6,2025-11-24,-2.45,0.2,3.758510,73.300674,munich


In [111]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Insert new data
#air_quality_fg.insert(combined_aq_df)

2025-11-13 23:32:33,706 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286301/fs/1273931/fg/1668839


Uploading Dataframe: 100.00% |██████████| Rows 4/4 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: air_quality_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286301/jobs/named/air_quality_3_offline_fg_materialization/executions


(Job('air_quality_3_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 739489
         }
       },
       "result": {
         "observed_value": 6.0,
         "element_count": 4,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-13T10:32:33.000706Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectati

In [112]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 13:59:05,217 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286301/fs/1273931/fg/1668709


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286301/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-18 13:59:22,273 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 13:59:25,485 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 14:00:45,662 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-11-18 14:00:48,862 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 14:00:49,027 INFO: Waiting for log aggregation to finish.
2025-11-18 14:01:04,389 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 739415
         }
       },
       "result": {
         "observed_value": 1.0800000429153442,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T12:59:05.000216Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type":

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
